In [ ]:
import glob 
import pandas as pd
import numpy as np
import os 
import json


In [ ]:
RAW_MATCH_DETAILS_DIR = "../Data/Raw/Match_details"

DNF_KEYWORDS = ['WO', 'INJ', 'RET', 'DSQ', 'DNS']
DNF_PATTERN = '|'.join(DNF_KEYWORDS)
DNF_PATTERN_CAPTURE = r'(' + r'|'.join(DNF_KEYWORDS) + r')'


# drop some columns that are not currently of interest for the project (e.g table number and venue))
# result status is "offical" for all entries - no neeed to keep
DROP_COLUMNS_START = [
    "resultStatus",        
    "playByPlaySequenceNumber"]

EVENTS_FILE = "../Data/Processed/Events/shortlist_events.csv"

CLEANED_MATCHES_DIR = "../Data/Processed/Matches"


JUNK_PATTERN_CAPTURE = r'([^\d,-]+)'
JUNK_PATTERN_CLEAN = r'[^\d,-]+'

In [ ]:
# Get all match files in the RAW_MATCH_DETAILS_DIR
# Create an all_matches_df to be filtered down 
# Many of these matches are in fact match ups between teams rather than players - need to filter out.
# some of the singles matches are from teams matches and events - can keep these.



# parse all match details inside the json files.
all_match_details_files = glob.glob(os.path.join(RAW_MATCH_DETAILS_DIR, "*.json"))
all_matches = []
for file in all_match_details_files:
    with open(file, 'r', encoding='utf-8') as f:
        matches_data = json.load(f)

    all_matches.extend(matches_data)

# create the df 

all_matches_df = pd.DataFrame(all_matches)







In [ ]:
# Keep here inside this cell to prevent rerunning file parsing and extraction every time.
all_matches_df = pd.DataFrame(all_matches)

# drop some columns that are not currently of interest for the project (e.g table number and venue etc )



# initialise the cleaned matches df and drop empty columns and na rows
print(f"cleaned_matches_df before  dropna and dropping irrelevant columns: {len(all_matches_df)} with {len(all_matches_df.columns)} cols")
cleaned_matches_df = all_matches_df
cleaned_matches_df.dropna(axis=0, how='all', inplace=True)
cleaned_matches_df.dropna(axis=1, how='all', inplace=True)
cleaned_matches_df.drop(columns=DROP_COLUMNS_START, inplace=True)
print(f"cleaned_matches_df after dropna and dropping irrelevant columns: {len(cleaned_matches_df)} with {len(cleaned_matches_df.columns)} cols")


dnf_from_overallScores = cleaned_matches_df["overallScores"].str.extract(JUNK_PATTERN_CAPTURE, expand=False).str.strip()
dnf_from_resultoverallScores = cleaned_matches_df["resultOverallScores"].str.extract(JUNK_PATTERN_CAPTURE, expand=False).str.strip()
dnf_from_gameScores = cleaned_matches_df["gameScores"].str.extract(JUNK_PATTERN_CAPTURE, expand=False).str.strip()
dnf_from_resultsGameScores = cleaned_matches_df["resultsGameScores"].str.extract(JUNK_PATTERN_CAPTURE, expand=False).str.strip()

cleaned_matches_df["overallScores"] = cleaned_matches_df["overallScores"].str.replace(JUNK_PATTERN_CLEAN, '', regex=True)
cleaned_matches_df["resultOverallScores"] = cleaned_matches_df["resultOverallScores"].str.replace(JUNK_PATTERN_CLEAN, '', regex=True)
cleaned_matches_df["gameScores"] = cleaned_matches_df["gameScores"].str.replace(JUNK_PATTERN_CLEAN, '', regex=True)
cleaned_matches_df["resultsGameScores"] = cleaned_matches_df["resultsGameScores"].str.replace(JUNK_PATTERN_CLEAN, '', regex=True)



cleaned_matches_df["dnf"] = dnf_from_overallScores
cleaned_matches_df["dnf"] = cleaned_matches_df["dnf"].fillna(dnf_from_resultoverallScores)
cleaned_matches_df["dnf"] = cleaned_matches_df["dnf"].fillna(False)

SCORE_RENAME_DICT = {
    # rename columns to A and B for better clarity
    "gameScores": "A_rawGameScores",
    "resultsGameScores": "B_rawGameScores",
    "overallScores": "A_rawOverallScore",
    "resultOverallScores": "B_rawOverallScore"
}

cleaned_matches_df.rename(columns=SCORE_RENAME_DICT, inplace=True)

print("✅ 'dnf' column updated by sequentially filling missing values from 'overallScores' THEN 'resultOverallScores'.")






In [ ]:
print(f"cleaned_df before dropping names with teams_parent_data: {len(cleaned_matches_df)}")
team_parent_filter = cleaned_matches_df["teamParentData"].notna()
team_parent_df  = cleaned_matches_df[team_parent_filter].copy()
cleaned_matches_df = cleaned_matches_df[~team_parent_filter ].copy()

# also drop team summaries
for col in cleaned_matches_df.columns:
    if "team" in col.lower():
        cleaned_matches_df.drop(columns=[col], inplace=True)


print(f"cleaned_df after dropping names with teams_parent_data: {len(cleaned_matches_df)}")

In [ ]:
print(f"cleaned_df before dropping para and age limit matches: {len(cleaned_matches_df)}")
age_limit_mask = cleaned_matches_df['subEventName'].str.contains(r"U\d{2}", case=False, na=False)
para_class_mask = cleaned_matches_df['subEventName'].str.contains("class", case=False, na=False)
age_para_filter = age_limit_mask | para_class_mask
cleaned_matches_df = cleaned_matches_df[~age_para_filter].copy()


print(f"cleaned_df after dropping para and age limit matches: {len(cleaned_matches_df)}")



In [ ]:
# Before filtering - extact key information from the 'competitors' column
# player name column can contain team names.
# competitors column keeps track of either the 1 player for a singles listing
# or the multiple players for a team listing 
# some doubles matches may be leftover from payload filtering - need to filter out.
# some doubles matches may be here inside team events - need to filter out.

def extract_competitor_details(competitor_list):
    """
    Extracts only the top-level competitor details (Name, ID, ORG) 
    for Home (H) and Away (A) competitors, ignoring the nested 'players' dict.
    """
    
    # Initialize the output dictionary  
    data = {}
    
    # check for empty data incase 
    if not isinstance(competitor_list, list) or len(competitor_list) < 2:
        return pd.Series(data)

    try:
        # use prefix pattern to determine home/away and build the keys for the output dict.
        for comp in competitor_list:
            comp_type = comp.get('competitorType')
            
            if comp_type == 'H':
                prefix = 'home'
            elif comp_type == 'A':
                prefix = 'away'
            else:
                continue                
           
            
            # get competitor id 
            data[f'{prefix}CompetitorId'] = comp.get('competitiorId', pd.NA)
                                
            # get competitor country code 
            data[f'{prefix}CompetitorOrg'] = comp.get('competitiorOrg', pd.NA)

            data[f'{prefix}Player'] = [player.get('playerName', pd.NA) for player in comp.get('players', pd.NA)]

            data[f'{prefix}NestedGameScores'] = comp.get('scores', pd.NA)



    except Exception as e:
        print(f"Error processing row: {e} | Data: {competitor_list}")
        pass

    return pd.Series(data)

#
print("--- 🚀 Getting competitor details 🚀 ---")

# apply the function to the competitors column from the main df
competitor_details_df = cleaned_matches_df['competitiors'].apply(extract_competitor_details)



cleaned_matches_df = pd.concat([cleaned_matches_df, competitor_details_df], axis=1)
cleaned_matches_df.drop(columns=["competitiors"],inplace=True, errors='ignore')
cleaned_matches_df["homePlayer"] = cleaned_matches_df["homePlayer"].str[0]
cleaned_matches_df["awayPlayer"] = cleaned_matches_df["awayPlayer"].str[0]



print("✅ Competitor details extracted and added to cleaned_matches_df and competitiors column dropped.")


In [ ]:
# All teams matches should be removed by now 

def extract_format(config):
    """
    Attempts to extract the best of format from the 'matchConfig' column
    """
    data = {"bestOf": pd.NA, "ttrReview": pd.NA}
    if not isinstance(config,dict):
        return pd.Series(data)
    try:
        data['bestOf'] = config.get('bestOfXGames')
        data['ttrReview'] = config.get('tTRReview')
        return pd.Series(data)
    except Exception as e:
        print(f"Error processing row: {e} | Data: {config}")
        pass
    return pd.Series(data)
#
print("--- 🚀 Getting Match Config deatils 🚀 ---")

# apply the function to the competitors column from the main df
match_config_df = cleaned_matches_df['matchConfig'].apply(extract_format)


cleaned_matches_df = pd.concat([cleaned_matches_df, match_config_df], axis=1)
cleaned_matches_df.drop(columns=["matchConfig"],inplace=True, errors='ignore')

print("✅ Match config extracted and added to cleaned_matches_df,matchConfig column dropped.")


In [ ]:
# serverNext = NEXT SERVER AFTER THE MATCH POINT WAS DONE
# (even if point would not be played as match was over)

def extract_next_server(action):
    """
    Attempts to extract the best of format from the 'matchConfig' column
    """
    data = {"serverNext":pd.NA,
            "actionType": pd.NA
}
    if not isinstance(action,dict):
        return pd.Series(action)
    try:
        
        data["serverNext"] = action.get("serverNext")   
        data["actionType"] = action.get("actionType")
        return pd.Series(data)
    except Exception as e:
        print(f"Error processing row: {e} | Data: {action}")
        pass
    return pd.Series(data)
#
print("--- 🚀 Getting Match Config deatils 🚀 ---")

# apply the function to the competitors column from the main df
last_server_df = cleaned_matches_df['action'].apply(extract_next_server)


cleaned_matches_df = pd.concat([cleaned_matches_df, last_server_df], axis=1)
cleaned_matches_df.drop(columns=["action"],inplace=True, errors='ignore')

print("✅ Match lastServer extracted and added to cleaned_matches_df,matchConfig column dropped.")

In [ ]:
def extract_times(matchTime):
    """
    Attempts to extract the match date and duration from the matchDate column
    """
    data = {"duration (unreliable)": pd.NA, "startDateLocal": pd.NA, "startDateUTC": pd.NA}
    if not isinstance(matchTime, dict):
        return pd.Series(matchTime)
    try:
        data['duration (unreliable)'] = matchTime.get('duration')
        data['startDateLocal'] = matchTime.get('startDateLocal')
        data['startDateUTC'] = matchTime.get('startDateUTC')
        return pd.Series(data)
    except Exception as e:
        print(f"Error processing row: {e} | Data: {matchTime}")
        pass
    return pd.Series(data)

times_df  = cleaned_matches_df['matchDateTime'].apply(extract_times)

cleaned_matches_df = pd.concat([cleaned_matches_df, times_df], axis=1)

cleaned_matches_df['startDateLocal'] = pd.to_datetime(
    cleaned_matches_df['startDateLocal'], errors='coerce', utc=False
)
cleaned_matches_df['startDateUTC'] = pd.to_datetime(
    cleaned_matches_df['startDateUTC'], errors='coerce', utc=True
)
cleaned_matches_df['startDateLocal'] = cleaned_matches_df['startDateLocal']
cleaned_matches_df['startDateUTC'] = cleaned_matches_df['startDateUTC']

cleaned_matches_df.drop(columns=["matchDateTime"],inplace=True, errors='ignore')

In [ ]:
events_df=pd.read_csv(EVENTS_FILE)
events_df["StartDateTime"] = pd.to_datetime(events_df["StartDateTime"], errors='coerce', utc=False)
events_df["EndDateTime"] = pd.to_datetime(events_df["EndDateTime"], errors='coerce', utc=False)

event_dates_df = events_df[["EventName","eventId", "StartDateTime"]]
event_dates_df = event_dates_df.rename(columns = {"StartDateTime":"EventStartDate"})
cleaned_matches_df["eventId"] = cleaned_matches_df["eventId"].astype(int)
cleaned_matches_df = cleaned_matches_df.merge(
    event_dates_df, 
    on='eventId', 
    how='left',
    validate='m:1'
)

In [ ]:

cleaned_matches_df = cleaned_matches_df.sort_values(by = ["EventStartDate", "startDateLocal", "matchStartTimeUTC"])

DATE_COLS = [
    "matchStartTimeUTC", # Already exists, just needs re-conversion for safety
    "startDateLocal",    # Needs conversion (was in nested dict)
    "startDateUTC",      # Needs conversion (was in nested dict)
    "EventStartDate"     # The merged date (likely a date object or string)
]


for col in DATE_COLS:
    # Check if the column exists in the DataFrame before trying to convert
    if col in cleaned_matches_df.columns:
        
        # 1. Convert to Datetime (errors='coerce' handles bad strings, turning them to NaT)
        # 2. Assign the result back to the same column
        cleaned_matches_df[col] = pd.to_datetime(
            cleaned_matches_df[col], 
            errors='coerce',
            utc=True # Ensure the resulting datetime object is timezone-aware (UTC)
        )


print("✅ All  date / timecolumns standardized to UTC datetime dtype.")

# Check the dtypes to confirm the conversion



In [ ]:
# One entry for muscat 2025 has a clearly erroneous date.
# For now, manually remove this date and let it be handled by fillNa hierarchy
target_event = all_matches_df["eventId"] == "3084"
target_match = all_matches_df["documentCode"] == "TTEWSINGLES-----------GP11000400----------"
target_mask = target_event & target_match
cleaned_matches_df.loc[target_mask,"matchStartTimeUTC"] = pd.NaT

In [ ]:
dates_hierarchy = ['matchStartTimeUTC',
                   'startDateUTC',
                   'startDateLocal',                  
                   'EventStartDate']

cleaned_matches_df['matchDate'] = cleaned_matches_df[dates_hierarchy[0]]
for col in dates_hierarchy[1:]:
    # This only fills rows where 'matchDate_filled' is currently NA
    cleaned_matches_df['matchDate'] = cleaned_matches_df['matchDate'].fillna(
        cleaned_matches_df[col]
    )
columns_to_drop = ['matchStartTimeUTC',
                   'startDateUTC',
                   'startDateLocal']
cleaned_matches_df = cleaned_matches_df.drop(columns=columns_to_drop, errors='ignore')


### Some matches start times are hours before their event start times - we can't fix that.
## not gonna break anything -i can't confirm which one is correct for each match / event.  

In [ ]:
def calculate_nested_scores(home_points_str, away_points_str):
    """
    Calculates the game scores and overall match score from point total strings.
    Returns: (calc_game_scores_str, calc_overall_scores_str)
    """
    # dont consider missing data 
    if pd.isna(home_points_str) or pd.isna(away_points_str):
        return pd.Series([pd.NA, pd.NA])
    
    # split strings into list 
    try:
        # Filter out empty strings and convert to int
        home_points = [int(p.strip()) for p in str(home_points_str).split(',') if p.strip()]
        away_points = [int(p.strip()) for p in str(away_points_str).split(',') if p.strip()]
    except ValueError:
        return pd.Series([pd.NA, pd.NA])

    # Ensure lists are the same length
    min_length = min(len(home_points), len(away_points))
    
    home_games_won = 0
    away_games_won = 0
    game_scores = []

    # zip home and away points - itererate over both up to length of shortest list 
    for h_pts, a_pts in zip(home_points[:min_length], away_points[:min_length]):
        
        # Skip 0-0 games
        if h_pts == 0 and a_pts == 0:
            continue
            
        # Create game score string 
        game_scores.append(f'{h_pts}-{a_pts}')
        
        # higher points tally as game wins
        if h_pts > a_pts:
            home_games_won += 1
        elif a_pts > h_pts:
            away_games_won += 1
            
    # join games scores intp output string
    calc_game_scores_str = ','.join(game_scores)
    calc_overall_scores_str = f'{home_games_won}-{away_games_won}'
    
    return pd.Series([calc_game_scores_str, calc_overall_scores_str])

print("--- 🚀 Calculating Game and Overall Scores from nested Home and Away game scores 🚀 ---")

# use .apply and lambdafunction onto whole df 
new_score_cols = cleaned_matches_df.apply(
    lambda row: calculate_nested_scores(row['homeNestedGameScores'], row['awayNestedGameScores']),
    axis=1,
    result_type='expand'
)

# Add names to the columns
new_score_cols.columns = ['calcNestedGameScores', 'calcNestedOverallScores']

# concat  new columns + original df 
cleaned_matches_df = pd.concat([cleaned_matches_df, new_score_cols], axis=1)
# cleaned_matches_df.drop(columns=["homePlayerGameScores","awayPlayerGameScores"],inplace=True,errors="ignore")

print("✅ Game scores and overall match scores calculated and added.")


In [ ]:
# normalise scores by removing 0-0 games (at end of scores i.e unplayed sets)
cleaned_matches_df["A_rawGameScores"] = cleaned_matches_df["A_rawGameScores"].str.replace(',0-0', '')
cleaned_matches_df["B_rawGameScores"] = cleaned_matches_df["B_rawGameScores"].str.replace(',0-0', '')

# clean all scores by removing entries that don't have non-0 digits



def clean_zero_scores(df: pd.DataFrame, score_column: str) -> pd.DataFrame:
    """
    Replaces score strings that contain NO digits from 1-9
    These entries are usually 0-0 fillers that should be removed
    to ease later processing.
    Args:
        df: The DataFrame to clean.
        score_column: The name of the column containing the game score strings.

    Returns:
        The DataFrame with the cleaned score column.
    """
    
    non_zero_score_mask = df[score_column].str.contains(r'[1-9]', regex=True, na=False)
    
    # Get the mask for rows that DO NOT contain non-zero digits
    mask = ~non_zero_score_mask    
   
    # replace the masked values with pd.NA
    df.loc[mask, score_column] = pd.NA

    # log to check it has worked :)
    
    print(f"✅ Replaced {mask.sum()} zero score strings in '{score_column}' with pd.NA.")
    
    return df

scores_columns = [col for col in cleaned_matches_df.columns if "score" in col.lower()]
for column in scores_columns:
    cleaned_matches_df = clean_zero_scores(cleaned_matches_df, column)
    


In [ ]:
#### Triangulate and reconcile incosistent overall scores ####


print("--- 🚀 reconciling overall scores 🚀 ---")


# strip remaining scores of strings before reconciling
cleaned_matches_df["calcNestedGameScores"] = cleaned_matches_df["calcNestedGameScores"].str.replace(JUNK_PATTERN_CLEAN, '', regex=True)
cleaned_matches_df["homeNestedGameScores"] = cleaned_matches_df["homeNestedGameScores"].str.replace(JUNK_PATTERN_CLEAN, '', regex=True)
cleaned_matches_df["awayNestedGameScores"] = cleaned_matches_df["awayNestedGameScores"].str.replace(JUNK_PATTERN_CLEAN, '', regex=True)

cleaned_matches_df["A_rawOverallScore"] = cleaned_matches_df["A_rawOverallScore"].str.replace(JUNK_PATTERN_CLEAN, '', regex=True)
cleaned_matches_df["B_rawOverallScore"] = cleaned_matches_df["B_rawOverallScore"].str.replace(JUNK_PATTERN_CLEAN, '', regex=True)





# normalise scores for string comparison, s1, s2, s3 = scores to be reconciled. 
s1 = cleaned_matches_df['calcNestedOverallScores'].astype(str).str.strip().replace({'nan': '', 'NaT': '', '<NA>': ''})
s2 = cleaned_matches_df['B_rawOverallScore'].astype(str).str.strip().replace({'nan': '', 'NaT': '', '<NA>': ''})
s3 = cleaned_matches_df['A_rawOverallScore'].astype(str).str.strip().replace({'nan': '', 'NaT': '', '<NA>': ''})

# use a temporary score_df to do work on the series
temp_scores_df = pd.DataFrame({'s1': s1, 's2': s2, 's3': s3}, index=cleaned_matches_df.index)

 
def check_overall_score_consistency(row):
    """
    Checks if all *present* scores in the row are identical.
    returns true if all scores agree, otherwise false
    """

    # temp score df only contains scores so just need to compare all rows
    valid_scores = set([s for s in row if s != ""])
    
    # if set is length 1 or 0 - scores are consistent. 
    return len(valid_scores) == 1

# apply the scores_check
scoreConsistenct = temp_scores_df.apply(check_overall_score_consistency, axis=1)


# Set dnf matches to be consistentScore = true, as these matches are a separate case.
is_dnf = cleaned_matches_df['dnf'] != False 

# create column in the orginal to flag inconsistent scores (for future reference if needed)
cleaned_matches_df['scoreConsistent'] = scoreConsistenct | is_dnf


# compare all scores - if 2 or more agree - that value is used
# otherwise scores are taken in order of the s1,s2,s3
def get_democratic_overall_score(row):
    valid_scores = [s for s in row if s!= ""]
    if not valid_scores: return ""
    mode_result = pd.Series(valid_scores).mode()
    
    if not mode_result.empty:
        return mode_result.iloc[0] 
    else:
        return row['s1'] # Fallback

cleaned_matches_df['reconciledOverallScore'] = temp_scores_df.apply(get_democratic_overall_score, axis=1)
# cleaned_matches_df.drop(columns=["resultOverallScores","overallScores", "calcOverallScores"],inplace=True, errors="ignore")

print("--- ✔️ Scores Triangulated / reconciliated ✔️ ---")

In [ ]:
cleaned_matches_df.fillna(value=pd.NA,inplace=True)

In [ ]:
# Check which scores are missing

missing_A_gameScores = cleaned_matches_df["A_rawGameScores"].isnull()
missing_B_gameScores = cleaned_matches_df["B_rawGameScores"].isnull() 
missing_calcNestedGameScores = cleaned_matches_df["calcNestedGameScores"].isnull()
not_dnf = cleaned_matches_df["dnf"] == False

missing_A_gameScores_df = cleaned_matches_df[missing_A_gameScores & not_dnf]
missing_B_gameScores_df = cleaned_matches_df[missing_B_gameScores & not_dnf]
missing_calcGameScores_df = cleaned_matches_df[missing_calcNestedGameScores & not_dnf]
print(f"Missing A_gameScores: {len(missing_A_gameScores_df)}")
print(f"Missing B_gameScores: {len(missing_B_gameScores_df)}")
print(f"Missing calcNestedGameScores: {len(missing_calcGameScores_df)}")


In [ ]:
cleaned_matches_df

In [ ]:
def get_overall_score_winner(row):

    try:    
        scores = row["reconciledOverallScore"]
        score_split = scores.split("-")
        home_score = int(score_split[0])
        away_score = int(score_split[1])
        
        if home_score > away_score:
            return "home"
        elif away_score > home_score:
            return "away"
        else:
            return "tie"
    except:
        return pd.NA
        


cleaned_matches_df["reconciledOverallScoreWinner"] = cleaned_matches_df.apply(get_overall_score_winner, axis=1)


In [ ]:
### Triangulate and reconcile incosistent overall scores ####


print("--- 🚀 reconciling game scores s 🚀 ---")





# normalise scores for string comparison
s1 = cleaned_matches_df['calcNestedGameScores'].astype(str).str.strip().replace({'nan': '', 'NaT': '', '<NA>': ''})
s2 = cleaned_matches_df['B_rawGameScores'].astype(str).str.strip().replace({'nan': '', 'NaT': '', '<NA>': ''})
s3 = cleaned_matches_df['A_rawGameScores'].astype(str).str.strip().replace({'nan': '', 'NaT': '', '<NA>': ''})

# use a temporary score_df to do work on the series
temp_scores_df = pd.DataFrame({'s1': s1, 's2': s2, 's3': s3}, index=cleaned_matches_df.index)

 
def check_score_consistency(row):
    """
    Checks if all available scores in the row are identical.
    returns true if all scores agree, otherwise false
    """

    # temp score df only contains scores so just need to compare all rows
    valid_scores = set([s for s in row if s != ""])
    
    # if set is length 1 or 0 - scores are consistent. 
    return len(valid_scores) == 1

# apply the scores_check
scoreConsistenct = temp_scores_df.apply(check_score_consistency, axis=1)


# Set dnf matches to be consistentScore = true, as these matches are a separate case.
is_dnf = cleaned_matches_df['dnf'] != False 

# create column in the orginal to flag inconsistent scores (for future reference if needed)
cleaned_matches_df['gameScoreConsistent'] = scoreConsistenct | is_dnf


# compare all scores - if 2 or more agree - that value is used
# otherwise scores are taken in order of the s1,s2,s3
def get_democratic_game_score(row):
    valid_scores = [s for s in row if s != ""]
    if not valid_scores: return ""
    mode_result = pd.Series(valid_scores).mode()
    
    if not mode_result.empty:
        return mode_result.iloc[0] 
    else:
        return row['s1'] # Fallback

cleaned_matches_df['reconciledGameScore'] = temp_scores_df.apply(get_democratic_game_score, axis=1)
# cleaned_matches_df.drop(columns=["resultOverallScores","overallScores", "calcOverallScores"],inplace=True, errors="ignore")

print("--- ✔️ Scores Triangulated / reconciliated ✔️ ---")

In [ ]:
def check_game_score_validity(score_str: str) -> str:
    """
    Checks if a single game score (e.g., '11-8' or '13-11') is valid.
    Returns a status string: 'Valid', 'Incomplete', or 'Invalid'.
    """
    try:
        home, away = map(int, score_str.split('-'))
    except ValueError:
        return "Invalid_Format"

    if home == 0 and away == 0:
        return "Incomplete"
    
    score_diff = abs(home - away)
    winner_score = max(home, away)

    if winner_score >= 11:
        if score_diff >= 2:
            return "Valid"
        else:
            return "Invalid_Too_Close" 
    else:
        return "Invalid_Too_Low" 
def calculate_reconciled_game_scores_winner(row) -> pd.Series:
# Use the cleaned game score string (e.g., '11-8,11-9,10-12')
    score_string = row.get("reconciledGameScore")

    # Initialize the outputs
    winner_result = "error"
    game_status_flags = []

    if not score_string:
        winner_result = pd.NA
    else:
        games = score_string.split(",")
        home_tally = 0
        away_tally = 0
        
        for game_score_pair in games:
            game_score_pair = game_score_pair.strip() # Clean up spaces
            if not game_score_pair:
                continue
                
            # Run the rule check for diagnostic column
            game_status = check_game_score_validity(game_score_pair)
            if game_status != "Valid":
                game_status_flags.append(f"{game_score_pair}:{game_status}")

            try:
                home_score, away_score = map(int, game_score_pair.split('-'))
            except ValueError:
                # If a score can't be parsed, stop tallying
                winner_result = "Error_Parsing"
                break
            
            # Use Python's match/case for clean winner determination logic
            match (home_score > away_score, away_score > home_score):
                case (True, False):
                    home_tally += 1
                case (False, True):
                    away_tally += 1
                case _:
                    continue

        # Final match winner determination (only if no parsing error occurred)
        if winner_result != "Error_Parsing":
            if home_tally > away_tally:
                winner_result = "home"
            elif away_tally > home_tally:
                winner_result = "away"
            else:
                winner_result = "tie"

    # --- RETURN A SERIES ---
    return pd.Series({
        'calculatedGameScoreWinner': winner_result,
        'gameScoreFlags': "; ".join(game_status_flags)
    })

new_cols_df = cleaned_matches_df.apply(calculate_reconciled_game_scores_winner, axis=1)

# Concat new data to original df 
cleaned_matches_df = pd.concat([cleaned_matches_df, new_cols_df], axis=1)



In [ ]:
# mark the flagged matches where a result is wrong 
flagged_matches = cleaned_matches_df["gameScoreFlags"]!= ""
not_dnf = cleaned_matches_df["dnf"] == False
winner_mismatch = (cleaned_matches_df["calculatedGameScoreWinner"] != cleaned_matches_df["reconciledOverallScoreWinner"])
score_errors_df  = cleaned_matches_df[(flagged_matches |  winner_mismatch) & not_dnf]

In [ ]:
score_errors_df.to_csv("../Data/Processed/Matches/score_errors.csv")


In [ ]:
score_errors_fixed_df = pd.read_csv("../Data/Processed/Matches/score_errors_AMENDED.csv", index_col=0)
STABLE_KEY = ['eventId', 'documentCode']

cleaned_matches_df = cleaned_matches_df.set_index(STABLE_KEY)
score_errors_fixed_df = score_errors_fixed_df.set_index(STABLE_KEY)
cleaned_matches_df.update(score_errors_fixed_df)
cleaned_matches_df = cleaned_matches_df.reset_index()



In [ ]:
cleaned_matches_df[winner_mismatch]

In [ ]:
cols_to_remake = ["gameScoreFlags", "calculatedGameScoreWinner","reconciledOverallScoreWinner"]
cleaned_matches_df.drop(columns=cols_to_remake, inplace=True,errors="ignore")
cleaned_matches_df["reconciledOverallScoreWinner"] = cleaned_matches_df.apply(get_overall_score_winner, axis=1)
new_cols_df = cleaned_matches_df.apply(calculate_reconciled_game_scores_winner, axis=1)
cleaned_matches_df = pd.concat([cleaned_matches_df, new_cols_df], axis=1)
flagged_matches = cleaned_matches_df["gameScoreFlags"]!= ""
not_dnf = cleaned_matches_df["dnf"] == False
winner_mismatch = (cleaned_matches_df["calculatedGameScoreWinner"] != cleaned_matches_df["reconciledOverallScoreWinner"])
score_errors_persistent_df = cleaned_matches_df[(flagged_matches | winner_mismatch) & not_dnf]

if len(score_errors_persistent_df) == 0:
    print("✅ All Errors Resolved")
    print(f"{len(score_errors_persistent_df)} Errors Remaining")
else:
    print(f"{len(score_errors_persistent_df)} Errors Remaining")


In [ ]:
def get_best_of(row):
    try:
        scores = row["reconciledOverallScore"].split("-")
        home_score = int(scores[0])
        away_score = int(scores[1])
        max_games_won = max(home_score, away_score)
    
        match max_games_won:
            case 4:
                return 7
            case 3:
                return 5
            case _:
                return 0
    except:
        return pd.NA
    

cleaned_matches_df["calcBestOf"] = cleaned_matches_df.apply(get_best_of, axis=1)

pd.to_numeric(cleaned_matches_df["calcBestOf"],errors="coerce")
pd.to_numeric(cleaned_matches_df["bestOf"],errors="coerce")

best_of_mismatch = cleaned_matches_df["calcBestOf"] != cleaned_matches_df["bestOf"]
bestOf_missing = cleaned_matches_df["bestOf"].isnull()



# Only mismatches are for Paris Olympics 2024 (data error) and Macau World cup 24 and 25 (special cases)
# The calcBestOf can be trusted it seems and taken as the source
# however macau group games shall be set to best of 0 
# macau world cup  2024 eventid = 2937, 2025 = 3109
# Paris Olympics Event ID  = 2603

In [ ]:
cleaned_matches_df["trueBestOf"]= cleaned_matches_df["calcBestOf"]





world_cup_filter = cleaned_matches_df["eventId"].isin([2937,3109])
world_cup_group_filter = cleaned_matches_df["documentCode"].str.contains("GP")
cleaned_matches_df.loc[world_cup_filter & world_cup_group_filter & not_dnf, "trueBestOf"] = 0

paris_olympics_filter = cleaned_matches_df["eventId"] == 2603
paris_olympics_singles_filter = cleaned_matches_df["subEventName"].str.contains("Singles")
cleaned_matches_df.loc[paris_olympics_filter & paris_olympics_singles_filter & not_dnf, "trueBestOf"] = int(7)



In [ ]:
cleaned_matches_df["trueBestOf"]     = pd.to_numeric(cleaned_matches_df["trueBestOf"],errors="coerce")
cleaned_matches_df["bestOf"]     = pd.to_numeric(cleaned_matches_df["bestOf"],errors="coerce")
best_of_mismatch = cleaned_matches_df["bestOf"] != cleaned_matches_df["trueBestOf"]
bestOf_missing = cleaned_matches_df[not_dnf]["trueBestOf"].isnull()

cleaned_matches_df[best_of_mismatch & ~bestOf_missing & ~world_cup_filter][["bestOf","trueBestOf","EventName"]]


In [ ]:
from datetime import date


date_string = date.today().strftime("%Y%m%d")
file_name = f"{date_string}_cleaned_matches.csv"
file_path = os.path.join(CLEANED_MATCHES_DIR, file_name)

cleaned_matches_df.to_csv(file_path, index=False)


In [ ]:
null_filter =cleaned_matches_df["reconciledOverallScore"].isnull() | cleaned_matches_df["reconciledGameScore"].isnull()
null_string_filter = (cleaned_matches_df["reconciledOverallScore"].str.len()==0) | (cleaned_matches_df["reconciledGameScore"].str.len()==0)
cleaned_matches_df[null_filter | null_string_filter]


In [ ]:
winner_mismatch = cleaned_matches_df["reconciledOverallScoreWinner"]!= cleaned_matches_df["calculatedGameScoreWinner"]
cleaned_matches_df[winner_mismatch]